# 以信息增益为特征选择的决策树算法（ID3）

## 创建数据
数据来自西瓜书

>     色泽,根蒂,敲声,纹理,脐部,触感,好瓜
- 青绿,蜷缩,浊响,清晰,凹陷,硬滑,是
- 乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,是
- 乌黑,蜷缩,浊响,清晰,凹陷,硬滑,是
- 青绿,蜷缩,沉闷,清晰,凹陷,硬滑,是
- 浅白,蜷缩,浊响,清晰,凹陷,硬滑,是
- 青绿,稍蜷,浊响,清晰,稍凹,软粘,是
- 乌黑,稍蜷,浊响,稍糊,稍凹,软粘,是
- 乌黑,稍蜷,浊响,清晰,稍凹,硬滑,是
- 乌黑,稍蜷,沉闷,稍糊,稍凹,硬滑,否
- 青绿,硬挺,清脆,清晰,平坦,软粘,否
- 浅白,硬挺,清脆,模糊,平坦,硬滑,否
- 浅白,蜷缩,浊响,模糊,平坦,软粘,否
- 青绿,稍蜷,浊响,稍糊,凹陷,硬滑,否
- 浅白,稍蜷,沉闷,稍糊,凹陷,硬滑,否
- 乌黑,稍蜷,浊响,清晰,稍凹,软粘,否
- 浅白,蜷缩,浊响,模糊,平坦,硬滑,否
- 青绿,蜷缩,沉闷,稍糊,稍凹,硬滑,否

1. 色泽>青绿:1, 乌黑:2, 浅白:3 
2. 根蒂>蜷缩:1, 稍蜷:2, 硬挺:3
3. 敲声>浊响:1,沉闷:2,清脆:3
4. 纹理>清晰:1，稍糊:2, 模糊:3
5. 脐部>凹陷:1,稍凹:2,平坦:3
6. 触感>硬滑:1,软粘:2

In [254]:
chn_data = [['青绿','蜷缩','浊响','清晰','凹陷','硬滑','是'],
['乌黑','蜷缩','沉闷','清晰','凹陷','硬滑','是'],
['乌黑','蜷缩','浊响','清晰','凹陷','硬滑','是'],
['青绿','蜷缩','沉闷','清晰','凹陷','硬滑','是'],
['浅白','蜷缩','浊响','清晰','凹陷','硬滑','是'],
['青绿','稍蜷','浊响','清晰','稍凹','软粘','是'],
['乌黑','稍蜷','浊响','稍糊','稍凹','软粘','是'],
['乌黑','稍蜷','浊响','清晰','稍凹','硬滑','是'],
['乌黑','稍蜷','沉闷','稍糊','稍凹','硬滑','否'],
['青绿','硬挺','清脆','清晰','平坦','软粘','否'],
['浅白','硬挺','清脆','模糊','平坦','硬滑','否'],
['浅白','蜷缩','浊响','模糊','平坦','软粘','否'],
['青绿','稍蜷','浊响','稍糊','凹陷','硬滑','否'],
['浅白','稍蜷','沉闷','稍糊','凹陷','硬滑','否'],
['乌黑','稍蜷','浊响','清晰','稍凹','软粘','否'],
['浅白','蜷缩','浊响','模糊','平坦','硬滑','否'],
['青绿','蜷缩','沉闷','稍糊','稍凹','硬滑','否']]

hash_dict =  {'青绿':1,'乌黑':2, '浅白':3, 
            '蜷缩':1, '稍蜷':2, '硬挺':3,
            '浊响':1,'沉闷':2,'清脆':3,
            '清晰':1,'稍糊':2, '模糊':3,
            '凹陷':1,'稍凹':2,'平坦':3,
            '硬滑':1,'软粘':2,
              '是':1,'否':0
             }

train = []
for row_vec in chn_data:
    tmp = []
    for col in row_vec:
        tmp.append(hash_dict[col])
    train.append(tmp)

feature_map = {0:'色泽',1:'根蒂',2:'敲声',3:'纹理',4:'脐部',5:'触感'}


In [219]:
import numpy as np

In [233]:
train = np.array(train)

## 算法推导 （ID3）
### Descision Tree through ID3

熵(entropy) 定义为：
$$P(X=x_i)=P_i, i\in{1,2,3...n},其中 1,2,3...为特征X_i的取值$$

随机变量X的熵定义为（表示这个随机变量的不确定性）
$$H(x)=-\sum_{i=1}^n{P_i*log{P_i}}$$

熵只依赖于$X$的分布,而与X的取值无关，so，可以写成
$$H(p)=-\sum_{i=1}^n{P_i*log{P_i}}$$

显然，熵值越大，随机变量(特征)的不确定性就越大，当然，在整个数据集中，单独看一个特征的熵是没有意义的，整个训练必定有标签$y$,那么就有$X,Y$的联合概率分布为：
$$P(X=x_i,Y=y_j)=P_{ij},    i\in{1,2,3...n},j\in{1,2,3..m}$$

如上，我们引入条件熵$H(Y|X)$,即表示已知随机变量$X$的条件下，$Y$的不确定性，换句话说，就是$X$对$Y$的某种影响有多大，这里的某种影响就是我们选择特征的重要来源之一：
$$H(Y|X)=\sum_{i=1}^n{P_i*H(Y|X=x_i)}$$

那么转换到实际的场景中，我们如何选择特征？=》信息增益，通俗的来讲，信息增益定义为：**已知$X$的信息使得类别$Y$不确定性减少的程度**，也就是我选择那个特征（$X$），能最大限度的分出类别$Y$，这就是信息增益要做的事情，那么信息增益定义为：

$$g(D,A)=H(D)-H(D|A)$$

其中$g(D,A)$表示在特征$A$下，$A$对整个数据集所产生的信息增益，$H(D)$为整个数据集的熵。

那么接下来可以计算数据集$D$的经验熵$H(D)$:

$$H(D)=-\sum_{k=1}^K{\frac{|C_k|}{|C|}}*{log\frac{|C_k|}{|C|}}$$

其中，$|D|$为样本总量，$|C_k|$为类别总量，接下来计算其条件经验熵(emprical entropy) $H(D|A)$:

$$H(D|A)=\sum_{i=1}^n{\frac{|D_i|}{|D|}}*H(D_i)=-\sum_{i=1}^n{\frac{|D_i|}{|D|}}*\sum_{k=1}^K\frac{|D_{ik}|}{|D_i|}*log{\frac{|D_{ik}|}{|D_i|}}$$

其中，$|D_i|$为对应的$A$特征下，取值为$i$的样本数，同理，$|D_{ik}|$就是在$D_i$的子数据下的分类类别，由此信息增益(info gain)可计算：

$$g(D,A)=H(D)-H(D|A)$$

NOTES:对数方面，一般以2为底，或取自然对数

In [221]:
def calculate_entropy(vector):
    """
    vector just contain the lables, 1D aray like so
    """
    elements = np.unique(vector)
    entropy = 0.0
    vec_size = vector.shape[0]
    
    for unique_val in elements:
        prob = list(vector).count(unique_val)/vec_size+0.0
        entropy -= prob*np.log2(prob)
    return entropy

In [222]:
def split_data(complete_data, feature_idx, val):
    """
    split the original data through feature index and its value
    then reurn the subset within label
    """
    subset = np.c_[complete_data[:,feature_idx], complete_data[:,-1]]
    subset_target = np.where(subset[:,0]==val)
    subset = complete_data[subset_target,:]
    subset = subset[0]
    return np.delete(subset, feature_idx, 1)

In [223]:
def info_gain(dataset, feature_idx):
    """
    calculate the appointed features's informathon gain while the dataset
    should store all the label, definitely.Otherwise,the programe will be wrong
    """
    base_entropy = calculate_entropy(dataset[:,-1])
    empircal_condition_entropy = 0.0
    
    for feature_val in np.unique(dataset[:, feature_idx]):
        subset = split_data(dataset, feature_idx, feature_val)
        current_entropy = calculate_entropy(subset[:,-1])
        empircal_condition_entropy += (subset.shape[0]/dataset.shape[0]+0.0)*current_entropy
    
    return base_entropy-empircal_condition_entropy

In [224]:
def chose_best_feature(dataset):
    maximum_infor_gain = 0.0
    best_feature_idx = -1
    
    for feature_idx in range(dataset.shape[1]-1):
        this_infor_gain = info_gain(dataset, feature_idx)
        if this_infor_gain>maximum_infor_gain:
            best_feature_idx = feature_idx
            maximum_infor_gain = this_infor_gain
    
    return best_feature_idx

In [252]:
def vote(classlist):
    return classlist[0]

In [253]:
def build_tree(train, feature_vec):
    
    classlist = list(train[:,-1])
    if classlist.count(classlist[0]) == len(classlist):
        return classlist[0]
    if train.shape[0]==1:
        
        return vote(classlist)
    
    best_feature_idx = chose_best_feature(train)
    best_feature_name = feature_vec[best_feature_idx]
    tree = {best_feature_name:{}}

    del(feature_vec[best_feature_idx])
    
    for feature_val in np.unique(train[:,best_feature_idx]):
        sub_feature_vec = feature_vec[:]
        tree[best_feature_name][feature_val] = build_tree(split_data(train, best_feature_idx, feature_val), sub_feature_vec)
    return tree


In [251]:
feature_vec = ['色泽','根蒂','敲声','纹理','脐部','触感']
build_tree(train, feature_vec)

{'纹理': {1: {'根蒂': {1: 1, 2: {'色泽': {1: 1, 2: {'触感': {1: 1, 2: 0}}}}, 3: 0}},
  2: {'触感': {1: 0, 2: 1}},
  3: 0}}